In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-urbanshift' 
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + '/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract images with population: total, with open space access and with tree access

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']

    # process aoi level ------
    # read boundaries
    boundary_path = aws_s3_dir +'/data/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #read open space
    openspace_path = aws_s3_dir +'/data/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    
    #load population
    pop = ee.ImageCollection('WorldPop/GP/100m/pop')
    pop = (pop.filter(
        ee.Filter.inList('year',[2020]))
        .select('population'))
    popImg = pop.mean().rename('population')

    popProj = pop.first().projection()
    popScale = popProj.nominalScale()

    # define threshold distance (m) and buffer open space areas by that to get recreation catchment
    DistanceThres = 400 # meters distance from population to be considered
    def amenityBuffer(feat):
      feat = ee.Feature(feat)
      return feat.buffer(DistanceThres)
    RecCatchment = openspace_geo_ee.map(amenityBuffer)
    RecCatchmentUnion = RecCatchment.union()
    
    #mask population by recreation catchment
    mask = ee.Image.constant(1).clipToCollection(RecCatchmentUnion).mask()
    popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
    
    ## add tree cover dataset
    TML = ee.ImageCollection('projects/wri-datalab/TML').filter(ee.Filter.bounds(boundary_geo_ee))
    TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
    
    # calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)
    TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 
    circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
    TreeCoverinThreshm = TreeCover.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)
    popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')
    
    # combine images 
    combImg = popImg.addBands([popwOSaccess,popwthresTC])
    
    
    geemap.ee_export_image_to_drive(
        popImg, 
        description = boundary_id_aoi + '-WorldPop-population',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )
    
    geemap.ee_export_image_to_drive(
        popwOSaccess, 
        description = boundary_id_aoi + '-population-wOpenSpace',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )
        
    geemap.ee_export_image_to_drive(
        popwthresTC, 
        description = boundary_id_aoi + '-population-wTreeCover',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)


In [ ]:
for i in range(0,len(boundary_georef)):
# for i in range(7,9):
# for i in range(11,22):
# for i in range(25,len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-WorldPop-population.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-WorldPop-population-2020.tif',
                               ExtraArgs={'ACL':'public-read'})

    # read local raster
    city_file = 'data/' + boundary_id +'-population-wTreeCover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-population-wTreeCover-2020.tif',
                               ExtraArgs={'ACL':'public-read'})
    
    # read local raster
    city_file = 'data/' + boundary_id +'-population-wOpenSpace.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-population-wOpenSpace-2020.tif',
                               ExtraArgs={'ACL':'public-read'})